# Requirements and Imports

Requirements installation

In [1]:
%pip install pm4py

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.3 MB/s eta 0:00:00
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26097 sha256=97028cb539458571979bd98a70201e88e8378ced2b8c91fa7e298195a127fec2
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built intervaltree


We make the required imports


In [2]:
import os
import shutil
from google.colab import files
import pm4py
import random
from datetime import datetime, timedelta
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.log.exporter.xes import exporter as xes_exporter

/usr/local/lib/python3.10/dist-packages/cudf/utils/_ptxcompiler.py:64: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/driver.py", line 295, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


Not patching Numba
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/cudf/utils/gpu_utils.py:62: UserWarning: Failed to dlopen libcuda.so.1
  warnings.warn(str(e))


# New Noise Settings



In [3]:
"""
Definition (Noise):
    Given a system trace `T` of length `n`, the noise `η(T)` is defined as the total number of constraint
    violations per unit length of the trace. Formally, if `C = {C_1, C_2, ..., C_m}` represents a set of behavioral
    constraints applied to the trace, then the noise `η(T)` is defined as:

    η(T) = (1 / n) * ∑_{i=1}^{m} V(C_i, T)

    where:
        - `n` is the length of the trace `T`.
        - `V(C_i, T)` is a function that counts the number of times the constraint `C_i` is violated in the trace `T`.

Interpretation:
    The noise `η(T)` quantifies the normalized number of instances in which any of the given constraints `C`
    are violated throughout the trace. This value provides an average rate of constraint violations per unit length,
    giving insight into how frequently the system behavior deviates from the expected or desired constraints relative
    to the length of the trace.The percentage is obtained in the usual way.
"""
# Alphabet 1
active_8_activities = {"a", "b", "c", "d", "e", "f", "g", "h", "i"}

# Alphabet 2
active_12_activities = {"a", "b", "c", "d", "e", "f", "g", "h", "i", "l", "m" ,"n"}

# Consistent declare model 1
active_constraints_8_activities = [
    "Not Existence:c",
    "Existence:a",
    "Not Chain Succession:b_b",
    "Existence:e",
    "Not Chain Succession:d_d",
    "Response:f_g",
    "Not Succession:h_i",
]

# Consistent declare model 2
active_constraints_12_activities = [
    "Not Existence:d",
    "Existence:a",
    "Not Chain Succession:b_c",
    "Response:m_n",
    "Not Chain Succession:e_f",
    "Not Succession:g_h",
    "Not Succession:i_l",
]

# Declare global variables
trace_length = 0
noise_percentage = 0
alphabet = 0
num_constraints = 0
noise_percentages = [0, 10, 20, 30]
file_path = ""

event_id_tag = "concept:name"
lifecycle_tag = "lifecycle:transition"
timestamp_tag = "time:timestamp"

# Global directories
current_dir = '/content/logs'
temp_dir = '/content/temp_zip'

# Synthetic Log Creation

In [4]:
def add_event(trace, event):
    new_trace = trace[:]
    position = random.randint(0, len(new_trace))
    lifecycle = "complete"
    start_date = datetime(2022, 3, 1)
    end_date = datetime(2023, 7, 31)
    random_timedelta = timedelta(seconds=random.randint(0, int((end_date - start_date).total_seconds())))
    random_timestamp = (start_date + random_timedelta).strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'
    new_event = {
        event_id_tag: event,
        lifecycle_tag: lifecycle,
        timestamp_tag: random_timestamp
    }
    new_trace.insert(position, new_event)
    trace[:] = new_trace


def delete_event(trace, event):
    if not act_in_trace(trace, [event]):
        return -1
    new_trace = [e for e in trace if e[event_id_tag] != event or e[lifecycle_tag] != "complete"]
    trace[:] = new_trace

def violate_response(trace, event, event2):
    if not act_in_trace(trace, [event,event2]):
        return -1
    new_trace = []
    event_found = False

    for e in trace:
        if event_found:
            if e[event_id_tag] == event2 and e[lifecycle_tag] == "complete":
                continue
        elif e[event_id_tag] == event and e[lifecycle_tag] == "complete":
            event_found = True

        new_trace.append(e)

    trace[:] = new_trace



def swap_events(trace, event_1, event_2):
    if not act_in_trace(trace, [event_1, event_2]):
        return -1
    indices = {e[event_id_tag]: i for i, e in enumerate(trace) if e[lifecycle_tag] == "complete"}
    new_trace = trace[:]
    if event_1 in indices and event_2 in indices and indices[event_1] > indices[event_2]:
        i, j = indices[event_1], indices[event_2]
        new_trace[i], new_trace[j] = new_trace[j], new_trace[i]
    else:
      return -1
    trace[:] = new_trace



def violate_chain_succession(trace, activity1, activity2):
    if not act_in_trace(trace, [activity1]):
        return -1
    new_trace = []
    for i in range(len(trace)):
        new_trace.append(trace[i])
        if trace[i][event_id_tag] == activity1 and trace[i][lifecycle_tag] == "complete":
            new_event = {
                event_id_tag: activity2,
                lifecycle_tag: "complete",
                timestamp_tag: trace[i][timestamp_tag]
            }
            new_trace.append(new_event)
            break
    if i < len(trace) - 1:
        new_trace.extend(trace[i + 1:])
    trace[:] = new_trace


def violate_constraint(trace, constraint, found_activities):
    if "Existence" in constraint and "Not" not in constraint:
        activity = constraint.split(":")[1]
        if not activity or activity not in found_activities:
            return -1
        if delete_event(trace, activity) == -1:
          return -1
    elif "Succession" in constraint and "Chain" not in constraint:
        activity1 = constraint.split(":")[1].split("_")[0]
        activity2 = constraint.split(":")[1].split("_")[1]
        if not activity1 or not activity2 or activity1 not in found_activities or activity2 not in found_activities:
            return -1
        if swap_events(trace, activity1, activity2) == -1:
          return -1
    elif "Response" in constraint:
        activity1 = constraint.split(":")[1].split("_")[0]
        activity2 = constraint.split(":")[1].split("_")[1]
        if not activity1 or not activity2 or activity2 not in found_activities or activity1 not in found_activities:
            return -1
        if violate_response(trace, activity1, activity2) == -1:
          return -1
    elif "Not Existence" in constraint:
        activity = constraint.split(":")[1]
        if not activity:
          return -1
        add_event(trace, activity)
    elif "Not Chain Succession" in constraint:
        activity1 = constraint.split(":")[1].split("_")[0]
        activity2 = constraint.split(":")[1].split("_")[1]
        if not activity1 or not activity2 or activity1 not in found_activities:
            return -1
        if violate_chain_succession(trace, activity1, activity2) == -1:
          return -1
    else:
        print(f"Unknown constraint: {constraint}")
        return -1
    return 0


def inject_noise(trace, active_constraints, noise_percentage, found_activities,num_total_violations,num_adds,num_deletes,num_swaps):
    violation_instances = 0
    finished = False
    while violation_instances != num_total_violations:
      # Inject add events
      for _ in range(num_adds):
          if not add_constraints:
              break
          constraint = random.choice(add_constraints)
          if violate_constraint(trace,constraint,found_activities) == 0:
              print(f"Constraint: {constraint}")
              violation_instances += 1
              if violation_instances == num_total_violations:
                finished = True
                break
      if finished:
        break
      # Inject delete events
      for _ in range(num_deletes):
          if not delete_constraints:
              break
          constraint = random.choice(delete_constraints)
          if violate_constraint(trace,constraint,found_activities) == 0:
            print(f"Constraint: {constraint}")
            violation_instances += 1
            if violation_instances == num_total_violations:
                finished = True
                break
      if finished:
        break

      # Inject swap events
      for _ in range(num_swaps):
          if not swap_constraints:
              break
          constraint = random.choice(swap_constraints)
          if violate_constraint(trace,constraint,found_activities) == 0:
            print(f"Constraint: {constraint}")
            violation_instances += 1
            if violation_instances == num_total_violations:
                finished = True
                break
      if finished:
        break

def act_in_trace(trace, activities):
    found_activities = {event[event_id_tag] for event in trace if event[lifecycle_tag] == "complete"}
    return set(activities).issubset(found_activities)

def activity_in_trace(trace, activity_in_trace):
    for event in trace:
      activity_in_trace.append(event[event_id_tag])

def inject_noise_into_log(log, active_constraints, noise_percentage):
    # Calculate the total number of violations needed based on the noise percentage
    num_total_violations = int((noise_percentage / 100) * trace_length)

    # Ensure the total violations are evenly split between add and delete
    if num_constraints != 1 and num_total_violations != 1:
        num_adds_and_deletes = (num_total_violations // 2) * 2
        num_adds = num_adds_and_deletes // 2
        num_deletes = num_adds
        num_swaps = num_total_violations - num_adds_and_deletes
    else:
        num_adds = num_total_violations
        num_deletes = 0
        num_swaps = 0

    print(f"Total Violations: {num_total_violations}")
    print(f"Adds: {num_adds}")
    print(f"Deletes: {num_deletes}")
    print(f"Swaps: {num_swaps}")
    for trace in log:
        found_activities = []
        activity_in_trace(trace, found_activities)
        original_length = len(trace)
        inject_noise(trace, active_constraints, noise_percentage, found_activities,num_total_violations,num_adds,num_deletes,num_swaps)
        new_length = len(trace)
        print(f"Same length: {original_length == new_length} original: {original_length} new:{new_length}")

# Log Formatting

In [5]:
def format_trace(trace):
    modified_trace = []
    start_date = datetime(2022, 3, 1)
    end_date = datetime(2023, 7, 31)

    random_timedelta = timedelta(seconds=random.randint(0, int((end_date - start_date).total_seconds())))
    timestamp = start_date + random_timedelta
    timestamp_str = timestamp.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'
    for event in trace:

        event_id = event[event_id_tag]
        event[timestamp_tag] = timestamp_str

        started_event = {
            event_id_tag: event_id,
            lifecycle_tag: 'start',
            timestamp_tag: timestamp_str
        }

        modified_trace.extend([started_event, event])

    trace[:] = modified_trace



def format_log(log):
    for index, trace in enumerate(log):
        trace.attributes[event_id_tag] = str(index)
        trace.attributes["trace:type"] = "pos"
        del trace.attributes["trace:type"]
        format_trace(trace)


# Execution

In [6]:
def main():
    global trace_length
    global noise_percentage
    global alphabet
    global num_constraints
    global file_path
    global active_constraints
    global add_constraints
    global delete_constraints
    global swap_constraints

    # Get the current working directory
    current_directory = "./"

    try:
        os.makedirs(current_dir, exist_ok=True)
        items = os.listdir(current_directory)
        for item in items:
            if os.path.isfile(item):
                print("Found item:", item)
                file_path = item
                try:
                    parts = item.split('_')
                    trace_length = int(parts[0])
                    noise_percentage = int(parts[1].replace('%', ''))
                    alphabet = int(parts[2])
                    num_constraints = int(parts[3].replace(".xes",""))
                except (IndexError, ValueError) as e:
                    print(f"Error parsing '{item}': {e}")
                    return -1

                print(f"##### Processing: {file_path} #####")
                print("Trace Length:", trace_length)
                print("Noise Percentage:", noise_percentage)
                print("Alphabet:", alphabet)
                print("Number of Constraints:", num_constraints)
                print("File Path:", file_path)

                active_activities = active_8_activities if alphabet == 8 else active_12_activities
                active_constraints_all = active_constraints_8_activities if alphabet == 8 else active_constraints_12_activities

                active_constraints = active_constraints_all[:num_constraints]

                add_constraints = [c for c in active_constraints if "Not Existence" in c or "Not Chain" in c]
                delete_constraints = [c for c in active_constraints if ("Existence" in c or "Response" in c) and (c not in add_constraints)]
                swap_constraints = [c for c in active_constraints if (c not in add_constraints) and (c not in delete_constraints)]

                print("Active Constraints:", active_constraints)
                print("Add Constraints:", add_constraints)
                print("Delete Constraints:", delete_constraints)
                print("Swap Constraints:", swap_constraints)

                noiser()

    except FileNotFoundError as e:
        print("Here")
        print(f"No matching file or directory found: {e}")
        return -1
    except Exception as e:
        print(f"An error occurred: {e}")
        return -1


def noiser():

    # Iterate over each combination of noise percentage for each type of log generated
    for np in noise_percentages:

        # Load the log
        log = xes_importer.apply(file_path)

        # Format the log
        format_log(log)

        # Inject noise into the log with the current constraints and percentage
        if np != 0:
            inject_noise_into_log(log, active_constraints, np)

        # Format filename
        output_filename = f"{current_dir}/{trace_length}_{np}%_{alphabet}_{num_constraints}.xes"

        # Save the noisy log to a new file
        xes_exporter.apply(log, output_filename)

        print(f"Log saved to {output_filename}")


main()

Found item: 10_0%_12_7.xes
##### Processing: 10_0%_12_7.xes #####
Trace Length: 10
Noise Percentage: 0
Alphabet: 12
Number of Constraints: 7
File Path: 10_0%_12_7.xes
Active Constraints: ['Not Existence:d', 'Existence:a', 'Not Chain Succession:b_c', 'Response:m_n', 'Not Chain Succession:e_f', 'Not Succession:g_h', 'Not Succession:i_l']
Add Constraints: ['Not Existence:d', 'Not Chain Succession:b_c', 'Not Chain Succession:e_f']
Delete Constraints: ['Existence:a', 'Response:m_n']
Swap Constraints: ['Not Succession:g_h', 'Not Succession:i_l']


/usr/local/lib/python3.10/dist-packages/pm4py/util/dt_parsing/parser.py:82: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  warnings.warn(


parsing log, completed traces ::   0%|          | 0/20 [00:00<?, ?it/s]

exporting log, completed traces ::   0%|          | 0/20 [00:00<?, ?it/s]

Log saved to /content/logs/10_0%_12_7.xes


...

Log saved to /content/logs/10_20%_12_5.xes


parsing log, completed traces ::   0%|          | 0/20 [00:00<?, ?it/s]

Total Violations: 3
Adds: 1
Deletes: 1
Swaps: 1
Constraint: Response:m_n
Constraint: Existence:a
Constraint: Not Existence:d
Same length: False original: 10 new:9
Constraint: Existence:a
Constraint: Not Existence:d
Constraint: Not Existence:d
Same length: False original: 10 new:11
Constraint: Not Chain Succession:e_f
Constraint: Response:m_n
Constraint: Not Existence:d
Same length: False original: 10 new:11
Constraint: Not Chain Succession:b_c
Constraint: Existence:a
Constraint: Not Existence:d
Same length: False original: 10 new:11
Constraint: Not Chain Succession:e_f
Constraint: Existence:a
Constraint: Not Existence:d
Same length: False original: 10 new:11
Constraint: Not Existence:d
Constraint: Existence:a
Constraint: Not Chain Succession:b_c
Same length: False original: 10 new:11
Constraint: Existence:a
Constraint: Response:m_n
Constraint: Not Existence:d
Same length: False original: 10 new:9
Constraint: Not Existence:d
Constraint: Not Existence:d
Constraint: Existence:a
Same lengt

exporting log, completed traces ::   0%|          | 0/20 [00:00<?, ?it/s]

Log saved to /content/logs/10_30%_12_5.xes


# Logs Exports

In this section we export as a zip the logs generated previously

In [7]:
# Step 1: Create the `current_dir` and ensure it exists
os.makedirs(current_dir, exist_ok=True)

# Step 2: List files in `current_dir`
files_in_dir = os.listdir(current_dir)

# Step 3: Create `temp_dir` for copying files
os.makedirs(temp_dir, exist_ok=True)

# Copy files from `current_dir` to `temp_dir`
for file_name in files_in_dir:
    file_path = os.path.join(current_dir, file_name)
    if os.path.isfile(file_path):
        shutil.copy(file_path, temp_dir)

# Step 4: Create a ZIP archive from `temp_dir`
shutil.make_archive(current_dir, 'zip', temp_dir)

# Step 5: Download the ZIP file
files.download(f'{current_dir}.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Cleaning

In [8]:
# Step 6: Clean up by removing `current_dir`, `temp_dir`, and the ZIP file
shutil.rmtree(current_dir)
shutil.rmtree(temp_dir)
os.remove(f'{current_dir}.zip')

# Step 7: Clear all files in the './' directory
for file_name in os.listdir('./'):
    file_path = os.path.join('./', file_name)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.remove(file_path)
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')